In [1]:
# Import necessary libraries

import pandas as pd
import requests


In [2]:
def eurostat_series(dataset, params, freq, value_name="value"):
    params = dict(params)
    params["format"] = "JSON"
    EUROSTAT_BASE = "https://ec.europa.eu/eurostat/api/dissemination/statistics/1.0/data"
    try:
        r = requests.get(f"{EUROSTAT_BASE}/{dataset}", params=params, timeout=30)
        if "sorry.ec.europa.eu" in r.url:
            raise ConnectionError(
                "Eurostat API unavailable (redirected to sorry.ec.europa.eu)."
            )
        r.raise_for_status()
        j = r.json()
    except (requests.RequestException, ValueError) as exc:
        raise ConnectionError(
            "Eurostat API unavailable or returned invalid JSON."
        ) from exc
    time_index = j["dimension"]["time"]["category"]["index"]
    values = j.get("value", {})
    df = (
        pd.DataFrame(
            [(t, values.get(str(i))) for t, i in time_index.items()],
            columns=["period", value_name],
        )
        .dropna()
        .sort_values("period")
    )
    periods = df.pop("period").astype(str)
    if freq == "M":
        periods = periods.str.replace(r"^(\d{4})M(\d{2})$", r"\1-\2", regex=True)
    df["time"] = (
        pd.PeriodIndex(periods, freq=freq).to_timestamp(how="end").normalize()
    )
    return df.reset_index(drop=True)


def eurostat_components(
    dataset,
    base_params,
    items,
    freq,
    dim="na_item",
    how="inner",
):
    if isinstance(items, (list, tuple)):
        items = {item: item for item in items}
    frames = []
    for item_code, col_name in items.items():
        params = dict(base_params, **{dim: item_code})
        frames.append(
            eurostat_series(dataset, params, freq, value_name=col_name)
        )
    if not frames:
        return pd.DataFrame(columns=["time"])
    df = frames[0]
    for frame in frames[1:]:
        df = df.merge(frame, on="time", how=how)
    return df.sort_values("time").reset_index(drop=True)


In [3]:
base_params = dict(geo="PT", unit="CP_MEUR")

# GDP and expenditure components (annual, current prices)
components = [
    "B1GQ",

    "P31_S14_S15",
    "P31_S13",
    "P32_S13",
    "P51G",
    "P52_P53",
    "P6",
    "P7",

    "D1",
    "B2A3G",
    "D2",
    "D3",

    "B1G",
    "D21",
    "D31",]

y = eurostat_components("nama_10_gdp", base_params, components, freq="Y")
y["gdp_eo_exp"] = y["B1GQ"] - (
    y["P31_S14_S15"]+
    y["P31_S13"]+
    y["P32_S13"]+
    y["P51G"]+y["P52_P53"]+
    y["P6"]-y["P7"])

y["gdp_eo_inc"] = y["B1GQ"] - (
    y["D1"]+
    y["B2A3G"]+
    y["D2"]-y["D3"])

y["gdp_eo_prod"] = y["B1GQ"] - (
    y["B1G"]+
    y["D21"]-y["D31"])

# B1G by industry (annual, current prices)
nace_components = [
    "TOTAL",
    "A",
    "B-E",
    "F",
    "G-I",
    "J",
    "K",
    "L",
    "M_N",
    "O-Q",
    "R-U",
]

b1g_by_nace = eurostat_components(
    "nama_10_a10",
    dict(base_params, na_item="B1G"),
    nace_components,
    freq="Y",
    dim="nace_r2",
    how="outer",
)


In [4]:
import plotly.graph_objects as go

df = y.dropna(subset=["B1GQ"]).sort_values("time").copy()
df["public_consumption"] = df["P31_S13"] + df["P32_S13"]
df["net_exports"] = df["P6"] - df["P7"]
df["time"] = df["time"] - pd.DateOffset(months=6)

components = [
    ("P31_S14_S15", "Consumo privado (P31_S14_S15)"),
    ("public_consumption", "Consumo público (P31_S13+P32_S13)"),
    ("P51G", "Investimento físico (P51G)"),
    ("P52_P53", "Variação de existências e valorações (P52_P53)"),
    ("net_exports", "Exportações líquidas (P6-P7)"),
]

colors = {
    "P31_S14_S15": "rgba(37, 99, 235, 0.6)",
    "public_consumption": "rgba(16, 185, 129, 0.6)",
    "P51G": "rgba(249, 115, 22, 0.6)",
    "P52_P53": "rgba(239, 68, 68, 0.6)",
    "net_exports": "rgba(107, 114, 128, 0.6)",
}

for col, _ in components:
    df[f"{col}_share"] = df[col] / df["B1GQ"] * 100

fig = go.Figure()
for col, label in components:
    fig.add_trace(
        go.Bar(
            x=df["time"],
            y=df[f"{col}_share"],
            name=label,
            marker_color=colors[col],
        )
    )

fig.update_layout(
    barmode="relative",
    template="plotly_white",
    title="Portugal: componentes do PIB pela despesa (% do PIB)",
    title_x=0.5,
    legend_title_text="",
    xaxis_title="",
    yaxis_title="",
    legend=dict(orientation="h", x=0.5, y=-0.2, xanchor="center", yanchor="top", bgcolor="rgba(0,0,0,0)", borderwidth=0),
)
fig.update_xaxes(dtick="M36", tickformat="%Y", hoverformat="%Y")
fig.update_yaxes(tickformat=".1f", ticksuffix="%")
fig.show()


In [5]:
import plotly.graph_objects as go

df = y.dropna(subset=["B1GQ"]).sort_values("time").copy()
df["D2_D3"] = df["D2"] - df["D3"]
df["time"] = df["time"] - pd.DateOffset(months=6)

components = [
    ("D1", "Remuneração dos empregados (D1)"),
    ("B2A3G", "Excedente bruto de exploração e rendimento misto (B2A3G)"),
    ("D2_D3", "Impostos líquidos sobre a produção e importações (D2-D3)"),
]

colors = {
    "D1": "rgba(37, 99, 235, 0.6)",
    "B2A3G": "rgba(16, 185, 129, 0.6)",
    "D2_D3": "rgba(249, 115, 22, 0.6)",
}

for col, _ in components:
    df[f"{col}_share"] = df[col] / df["B1GQ"] * 100

fig = go.Figure()
for col, label in components:
    fig.add_trace(
        go.Bar(
            x=df["time"],
            y=df[f"{col}_share"],
            name=label,
            marker_color=colors[col],
        )
    )

fig.update_layout(
    barmode="relative",
    template="plotly_white",
    title="Portugal: componentes do PIB pela ótica do rendimento (% do PIB)",
    title_x=0.5,
    legend_title_text="",
    xaxis_title="",
    yaxis_title="",
    legend=dict(orientation="h", x=0.5, y=-0.2, xanchor="center", yanchor="top", bgcolor="rgba(0,0,0,0)", borderwidth=0),
)
fig.update_xaxes(dtick="M36", tickformat="%Y", hoverformat="%Y")
fig.update_yaxes(tickformat=".1f", ticksuffix="%")
fig.show()


In [6]:
import plotly.graph_objects as go

df = (
    b1g_by_nace.merge(
        y[["time", "B1GQ", "D21", "D31"]],
        on="time",
        how="inner",
    )
    .sort_values("time")
    .copy()
)

df["net_taxes"] = df["D21"] - df["D31"]
df["time"] = df["time"] - pd.DateOffset(months=6)

sector_components = [
    c for c in nace_components if c != "TOTAL" and c in df.columns
]

sector_labels = {
    "A": "Agric., silvic. e pesca (A)",
    "B-E": "Indústria (B-E)",
    "F": "Construção (F)",
    "G-I": "Comércio, transp. e aloj. (G-I)",
    "J": "Informação e comunicação (J)",
    "K": "Atividades fin. e de seguros (K)",
    "L": "Atividades imobiliárias (L)",
    "M_N": "Atividades prof. e admin. (M_N)",
    "O-Q": "Admin. púb., educ. e saúde (O-Q)",
    "R-U": "Artes e outros serviços (R-U)",
}

components = [(c, sector_labels.get(c, c)) for c in sector_components]
components.append(("net_taxes", "Impostos líq. nos prod. (D21-D31)"))

palette = [
    "rgba(37, 99, 235, 0.6)",
    "rgba(16, 185, 129, 0.6)",
    "rgba(249, 115, 22, 0.6)",
    "rgba(239, 68, 68, 0.6)",
    "rgba(107, 114, 128, 0.6)",
    "rgba(14, 165, 233, 0.6)",
    "rgba(168, 85, 247, 0.6)",
    "rgba(234, 179, 8, 0.6)",
    "rgba(20, 184, 166, 0.6)",
    "rgba(236, 72, 153, 0.6)",
    "rgba(148, 163, 184, 0.6)",
]

palette = palette * ((len(components) // len(palette)) + 1)
colors = {c: palette[i] for i, (c, _) in enumerate(components)}

for col, _ in components:
    df[f"{col}_share"] = df[col] / df["B1GQ"] * 100

fig = go.Figure()
for col, label in components:
    fig.add_trace(
        go.Bar(
            x=df["time"],
            y=df[f"{col}_share"],
            name=label,
            marker_color=colors[col],
        )
    )

fig.update_layout(
    barmode="relative",
    template="plotly_white",
    title="Portugal: componentes do PIB pela ótica da produção (% do PIB)",
    title_x=0.5,
    legend_title_text="",
    xaxis_title="",
    yaxis_title="",
    legend=dict(x=1.02, y=0.5, xanchor="left", yanchor="middle", bgcolor="rgba(0,0,0,0)", borderwidth=0),
)
fig.update_xaxes(dtick="M36", tickformat="%Y", hoverformat="%Y")
fig.update_yaxes(tickformat=".1f", ticksuffix="%")
fig.show()


In [7]:
years = [2004, 2014, 2024]


def series_by_year(df, col, years):
    tmp = df.copy()
    tmp["year"] = tmp["time"].dt.year
    tmp = tmp[tmp["year"].isin(years)]
    return tmp.set_index("year")[col].reindex(years)


def add_row(rows, approach, name, code, values, years):
    row = {
        "Abordagem": approach,
        "Componente": name,
        "Código": code,
    }
    for yr in years:
        row[str(yr)] = values.loc[yr]
    rows.append(row)


gdp_series = series_by_year(y[["time", "B1GQ"]], "B1GQ", years)

rows = []
add_row(rows, "-", "PIB a preços correntes", "B1GQ", gdp_series, years)

exp_df = y[
    [
        "time",
        "B1GQ",
        "P31_S14_S15",
        "P31_S13",
        "P32_S13",
        "P51G",
        "P52_P53",
        "P6",
        "P7",
    ]
].copy()
exp_df["public_consumption"] = exp_df["P31_S13"] + exp_df["P32_S13"]
exp_df["net_exports"] = exp_df["P6"] - exp_df["P7"]

exp_components = [
    ("P31_S14_S15", "Consumo privado", "P31_S14_S15"),
    ("public_consumption", "Consumo público", "P31_S13+P32_S13"),
    ("P51G", "Investimento físico", "P51G"),
    ("P52_P53", "Variação de existências e valorações", "P52_P53"),
    ("net_exports", "Exportações líquidas", "P6-P7"),
]

for col, name, code in exp_components:
    values = series_by_year(exp_df, col, years) / gdp_series * 100
    add_row(rows, "Despesa", name, code, values, years)

inc_df = y[["time", "B1GQ", "D1", "B2A3G", "D2", "D3"]].copy()
inc_df["D2_D3"] = inc_df["D2"] - inc_df["D3"]

inc_components = [
    ("D1", "Remuneração dos empregados", "D1"),
    (
        "B2A3G",
        "Excedente bruto de exploração e rendimento misto",
        "B2A3G",
    ),
    (
        "D2_D3",
        "Impostos líquidos sobre a produção e importações",
        "D2-D3",
    ),
]

for col, name, code in inc_components:
    values = series_by_year(inc_df, col, years) / gdp_series * 100
    add_row(rows, "Rendimento", name, code, values, years)

prod_df = (
    b1g_by_nace.merge(
        y[["time", "B1GQ", "D21", "D31"]],
        on="time",
        how="inner",
    )
    .sort_values("time")
    .copy()
)
prod_df["net_taxes"] = prod_df["D21"] - prod_df["D31"]

sector_components = [
    c for c in nace_components if c != "TOTAL" and c in prod_df.columns
]

sector_labels = {
    "A": "Agricultura, silvicultura e pesca",
    "B-E": "Indústria",
    "F": "Construção",
    "G-I": "Comércio, transportes e alojamento",
    "J": "Informação e comunicação",
    "K": "Atividades financeiras e de seguros",
    "L": "Atividades imobiliárias",
    "M_N": "Atividades profissionais e administrativas",
    "O-Q": "Administração pública, educação e saúde",
    "R-U": "Artes e outros serviços",
}

for code in sector_components:
    name = sector_labels.get(code, code)
    values = series_by_year(prod_df, code, years) / gdp_series * 100
    add_row(rows, "Produção", name, code, values, years)

values = series_by_year(prod_df, "net_taxes", years) / gdp_series * 100
add_row(
    rows,
    "Produção",
    "Impostos líquidos sobre os produtos",
    "D21-D31",
    values,
    years,
)

table = pd.DataFrame(rows)
table = table.round(1)
table


,Abordagem,Componente,Código,2004,2014,2024
0,-,PIB a preços correntes,B1GQ,152248.4,173186.7,289428.0
1,Despesa,Consumo privado,P31_S14_S15,63.7,66.1,60.9
2,Despesa,Consumo público,P31_S13+P32_S13,20.4,18.4,16.9
3,Despesa,Investimento físico,P51G,23.4,15.1,20.4
4,Despesa,Variação de existências e valorações,P52_P53,0.4,0.2,-0.1
5,Despesa,Exportações líquidas,P6-P7,-7.9,0.1,1.8
6,Rendimento,Remuneração dos empregados,D1,47.5,44.3,47.7
7,Rendimento,Excedente bruto de exploração e rendimento misto,B2A3G,40.4,42.9,39.2
8,Rendimento,Impostos líquidos sobre a produção e importações,D2-D3,12.1,12.8,13.1
9,Produção,"Agricultura, silvicultura e pesca",A,2.6,2.0,2.0


In [8]:
# Labor income share (annual)
labor_params = dict(geo="PT", unit="THS_PER")

labor_inputs = eurostat_components(
    "nama_10_pe",
    labor_params,
    ["EMP_DC", "SAL_DC"],
    freq="Y",
)

labor_share = (
    y.merge(labor_inputs, on="time", how="inner")
    .sort_values("time")
    .copy()
)

labor_share["GVAfp"] = labor_share["B1GQ"] - (
    labor_share["D2"] - labor_share["D3"]
)
labor_share["LIS_na"] = labor_share["D1"] / labor_share["GVAfp"]
labor_share["LIS_a"] = labor_share["LIS_na"] * (
    labor_share["EMP_DC"] / labor_share["SAL_DC"]
)
labor_share["LIS_bad"] = labor_share["D1"] / labor_share["B1GQ"]

labor_share = labor_share[
    [
        "time",
        "B1GQ",
        "D1",
        "D2",
        "D3",
        "GVAfp",
        "EMP_DC",
        "SAL_DC",
        "LIS_na",
        "LIS_a",
        "LIS_bad",
    ]
]


In [9]:
import plotly.express as px

df = (
    labor_share[["time", "LIS_na", "LIS_a", "LIS_bad"]]
    .dropna()
    .sort_values("time")
    .copy()
)
df["time"] = df["time"] - pd.DateOffset(months=6)

df_long = (
    df.rename(
        columns={
            "LIS_na": "não ajustada: D1/GVAfp",
            "LIS_a": "ajustada: (D1/GVAfp)(EMP/SAL)",
            "LIS_bad": "ingénua/incorrecta: D1/PIB",
        }
    )
    .melt(
        id_vars="time",
        value_vars=[
            "ajustada: (D1/GVAfp)(EMP/SAL)",
            "não ajustada: D1/GVAfp",
            "ingénua/incorrecta: D1/PIB",
        ],
        var_name="series",
        value_name="lis",
    )
)

df_long["lis"] = df_long["lis"] * 100

fig = px.line(
    df_long,
    x="time",
    y="lis",
    color="series",
    title="Portugal: parte do rendimento do trabalho (labor share)",
    template="plotly_white",
    color_discrete_map={
        "não ajustada: D1/GVAfp": "red",
        "ajustada: (D1/GVAfp)(EMP/SAL)": "blue",
        "ingénua/incorrecta: D1/PIB": "orange",
    },
    markers=True,
    line_shape="linear",
)

fig.update_traces(
    line_width=2.5,
    marker_size=6,
    marker_color="white",
    marker_symbol="circle",
    marker_line_width=2.0,
)

fig.update_layout(
    title_x=0.5,
    legend_title_text="",
    xaxis_title="",
    yaxis_title="",
    legend=dict(
        x=0.98,
        y=0.02,
        xanchor="right",
        yanchor="bottom",
        bgcolor="rgba(0,0,0,0)",
        borderwidth=0,
    ),
)
fig.update_xaxes(dtick="M36", tickformat="%Y", hoverformat="%Y")
fig.update_yaxes(tickformat=".1f", ticksuffix="%", range=[25, 85])
fig.show()


In [10]:
# OECD symmetric input-output table (IOTs 2021 edition)
import xml.etree.ElementTree as ET

base = "https://sdmx.oecd.org/archive/rest"
dataflow = "DF_IOTS_2021"


def fetch_codes(codelist_id):
    url = f"{base}/codelist/OECD/{codelist_id}/1.0"
    resp = requests.get(url, timeout=30)
    resp.raise_for_status()
    root = ET.fromstring(resp.text)
    ns = {
        "structure": "http://www.sdmx.org/resources/sdmxml/schemas/v2_1/structure",
        "common": "http://www.sdmx.org/resources/sdmxml/schemas/v2_1/common",
    }
    return [
        code.get("id")
        for code in root.findall(".//structure:Code", ns)
        if code.get("id") is not None
    ]


def sdmx_generic_to_df(xml_text):
    root = ET.fromstring(xml_text)
    ns = {
        "generic": "http://www.sdmx.org/resources/sdmxml/schemas/v2_1/data/generic",
    }
    rows = []
    for series in root.findall(".//generic:Series", ns):
        key = {
            v.get("id"): v.get("value")
            for v in series.findall("./generic:SeriesKey/generic:Value", ns)
        }
        for obs in series.findall("./generic:Obs", ns):
            time = obs.find("./generic:ObsDimension", ns).get("value")
            val_elem = obs.find("./generic:ObsValue", ns)
            val = float(val_elem.get("value")) if val_elem is not None else None
            rows.append({**key, "TIME_PERIOD": time, "value": val})
    return pd.DataFrame(rows)


col_codes = [c for c in fetch_codes("CL_IOTS_2021__COL") if c.startswith("D")]
row_codes_all = set(fetch_codes("CL_IOTS_2021__ROW"))
row_codes = [
    f"TTL_{c[1:]}" for c in col_codes if f"TTL_{c[1:]}" in row_codes_all
]

sample_url = f"{base}/data/{dataflow}/TTL.PRT.TTL_01T02.D01T02"
sample = requests.get(sample_url, timeout=30)
sample.raise_for_status()
sample_df = sdmx_generic_to_df(sample.text)
latest_year = str(sample_df["TIME_PERIOD"].astype(int).max())

row_key = "+".join(row_codes)
col_key = "+".join(col_codes)

data_url = f"{base}/data/{dataflow}/TTL.PRT.{row_key}.{col_key}.{latest_year}"
resp = requests.get(data_url, timeout=30)
resp.raise_for_status()

df = sdmx_generic_to_df(resp.text)
df = df[df["TIME_PERIOD"] == latest_year].copy()

siot_oecd = (
    df.pivot_table(
        index="ROW",
        columns="COL",
        values="value",
        aggfunc="first",
    )
    .reindex(index=row_codes, columns=col_codes)
    .copy()
)
siot_oecd.index = [c.replace("TTL_", "D") for c in siot_oecd.index]


In [11]:
# OECD IOTs Total Table (latest Portugal)
import io
import re
import zipfile

readme_url = "https://stats.oecd.org/wbos/fileview2.aspx?IDFile=c176b280-717a-42c0-acab-26e06d0137de"
zip_url = "https://stats.oecd.org/wbos/fileview2.aspx?IDFile=67d903bc-edff-463c-aeeb-52d711731221"

readme_resp = requests.get(readme_url, timeout=60)
readme_resp.raise_for_status()
iot_readme = pd.read_excel(io.BytesIO(readme_resp.content), sheet_name=None)

zip_resp = requests.get(zip_url, timeout=60)
zip_resp.raise_for_status()

z = zipfile.ZipFile(io.BytesIO(zip_resp.content))
prt_files = [name for name in z.namelist() if name.startswith("PRT") and name.endswith("ttl.csv")]
if not prt_files:
    raise ValueError("No PRT ttl files found in IOTs_TTL.zip")

years = []
for name in prt_files:
    match = re.search(r"(\d{4})", name)
    if match:
        years.append(int(match.group(1)))
latest_year = max(years)
latest_file = [name for name in prt_files if str(latest_year) in name][0]

with z.open(latest_file) as f:
    iot_prt_ttl = pd.read_csv(f, index_col=0)

iot_prt_ttl.index.name = "ROW"
iot_prt_year = latest_year
iot_prt_file = latest_file


In [12]:
# Aggregate the OECD Total IOT (TTL) to NACE letter level (A..T)
final_demand_cols = [
    "HFCE",
    "NPISH",
    "GGFC",
    "GFCF",
    "INVNT",
    "DPABR",
    "CONS_NONRES",
    "EXPO",
    "IMPO",
    "TOTAL",
]

industry_cols = [c for c in iot_prt_ttl.columns if c not in final_demand_cols]
industry_col_groups = [c[0] for c in industry_cols]

# First aggregate industry columns to letters; keep final demand columns as-is.
iot_cols_letter = pd.concat(
    [
        iot_prt_ttl[industry_cols].T.groupby(industry_col_groups, sort=False).sum().T,
        iot_prt_ttl[final_demand_cols],
    ],
    axis=1,
)

iot_cols_letter.index.name = "ROW"

# Then aggregate industry rows (TTL_*) to letters.
industry_rows = [
    r for r in iot_cols_letter.index if r.startswith("TTL_") and r != "TTL_INT_FNL"
]

ttl = iot_cols_letter.loc[industry_rows].copy()
ttl.index = [r.replace("TTL_", "")[0] for r in ttl.index]
ttl = ttl.groupby(level=0).sum()

other = iot_cols_letter.drop(index=industry_rows)

iot_prt_ttl_letter = pd.concat([ttl, other], axis=0)



In [13]:
# Aggregate letter-level industries into broad sectors
import re

sector_order = ["A", "B-E", "F", "G-I", "J", "K", "L", "M-N", "O-Q", "R-U"]

letter_to_sector = {
    "A": "A",
    "B": "B-E",
    "C": "B-E",
    "D": "B-E",
    "E": "B-E",
    "F": "F",
    "G": "G-I",
    "H": "G-I",
    "I": "G-I",
    "J": "J",
    "K": "K",
    "L": "L",
    "M": "M-N",
    "N": "M-N",
    "O": "O-Q",
    "P": "O-Q",
    "Q": "O-Q",
    "R": "R-U",
    "S": "R-U",
    "T": "R-U",
    "U": "R-U",
}

industry_letters = [c for c in iot_prt_ttl_letter.columns if re.fullmatch(r"[A-Z]", str(c))]
letter_groups = [letter_to_sector.get(c, c) for c in industry_letters]

cols_grouped = iot_prt_ttl_letter[industry_letters].T.groupby(letter_groups, sort=False).sum().T
other_cols = iot_prt_ttl_letter.drop(columns=industry_letters)

iot_cols_grouped = pd.concat([cols_grouped, other_cols], axis=1)

industry_rows = [r for r in iot_cols_grouped.index if re.fullmatch(r"[A-Z]", str(r))]
rows = iot_cols_grouped.loc[industry_rows].copy()
rows.index = [letter_to_sector.get(r, r) for r in rows.index]
rows = rows.groupby(level=0, sort=False).sum()
other_rows = iot_cols_grouped.drop(index=industry_rows)

iot_prt_ttl_broad = pd.concat([rows, other_rows], axis=0)

col_order = [s for s in sector_order if s in iot_prt_ttl_broad.columns]
other_col_order = [c for c in iot_prt_ttl_broad.columns if c not in col_order]
iot_prt_ttl_broad = iot_prt_ttl_broad[col_order + other_col_order]

row_order = [s for s in sector_order if s in iot_prt_ttl_broad.index]
other_row_order = [r for r in iot_prt_ttl_broad.index if r not in row_order]
iot_prt_ttl_broad = iot_prt_ttl_broad.loc[row_order + other_row_order]

iot_prt_ttl_broad


,A,B-E,F,G-I,J,K,L,M-N,O-Q,R-U,HFCE,NPISH,GGFC,GFCF,INVNT,DPABR,CONS_NONRES,EXPO,IMPO,TOTAL
A,2234.660,6799.511,84.198,933.982,13.487,6.768,11.483,54.587,97.975,25.307,5297.913,0.419,8.275,565.885,10.498,35.310,135.730,1767.210,-4413.596,13669.601
B-E,3465.964,81375.615,9333.683,17540.632,1422.445,506.835,743.667,2502.481,6014.812,1237.926,44253.483,11.882,1137.691,11237.780,1078.319,274.188,1173.343,61464.613,-81129.776,163645.606
F,192.792,1623.931,5637.760,1186.825,156.242,95.612,1314.822,431.832,945.650,106.795,252.783,0.113,236.413,19946.611,1.225,11.609,28.861,54.420,-139.969,32084.323
G-I,1730.971,20877.037,2896.283,14539.378,1050.634,638.596,278.522,2252.191,3164.587,692.441,35692.113,60.490,2746.040,4550.722,144.827,1969.026,12925.127,22010.142,-22195.051,106024.070
J,52.468,1705.282,258.126,2353.003,4442.293,808.681,174.984,1878.624,1143.439,371.073,3667.664,9.749,288.533,4837.980,18.958,97.503,394.634,4644.947,-6050.371,21097.573
K,288.373,2961.746,401.349,3026.785,411.463,3055.535,1156.862,986.005,673.645,270.194,6062.669,5.656,281.566,283.896,4.139,70.164,351.580,1141.238,-2101.016,19331.848
L,21.320,684.248,187.132,2728.378,344.771,256.095,326.782,563.643,531.188,208.536,20974.530,0.189,60.817,4013.786,0.120,304.988,1077.216,30.660,-357.459,31956.944
M-N,373.012,7090.527,1852.188,7967.973,2061.841,1528.757,881.228,6214.011,2853.636,892.547,2327.858,21.462,317.642,2165.961,19.387,89.712,369.898,6441.863,-6326.497,37143.009
O-Q,38.159,595.054,180.198,652.678,166.854,118.907,79.468,371.232,1902.923,81.190,11923.803,3265.778,38782.109,850.853,1.580,335.095,771.728,462.889,-666.958,59913.541
R-U,17.262,242.362,51.890,354.799,151.316,56.034,21.170,186.498,185.723,831.970,6550.397,1342.486,368.620,153.764,1.520,129.371,817.804,200.625,-528.185,11135.423


Exported OECD IOTs Portugal TTL broad to OECD_IOTs_PRT_TTL_broad.xlsx
